In [1]:
import pandas as pd

In [2]:
cols_to_use = ["id", "source", "name", "description", "neighborhood_overview", "host_id", "host_name", "host_since", "host_location",
               "host_about", "host_response_rate", "host_acceptance_rate", "host_is_superhost", "host_verifications", "host_has_profile_pic", 
               "host_identity_verified", "neighbourhood_group_cleansed", "property_type", "room_type", "accommodates", "bathrooms", "bathrooms_text", 
               "bedrooms", "beds", "amenities", "price", "minimum_nights", "maximum_nights", "availability_365", "number_of_reviews",
               "instant_bookable", "calculated_host_listings_count"]

# Sort only according to the columns we want
df= pd.read_csv("./listings.csv", sep = ',', usecols = cols_to_use)

In [3]:
# Function to convert text descriptions to length (number of characters)
def text_to_len(df: pd.DataFrame, text_columns: list):
    for col in text_columns:
        df.loc[:,col] = df[col].apply(lambda x: len(str(x).split()) if isinstance(x, str) else 0)
    return df

# Relevant for ["name", "description", "neighborhood_overview", "host_about, "host_verifications", "amenities"]

In [4]:
# Function to convert host_since dates to datetime to instead have a column representing how many days they have been host
def days_as_host(df: pd.DataFrame, date_col: str, reference_date: str):
    reference_date = pd.Timestamp(reference_date)
    df[date_col] = pd.to_datetime(df[date_col]) # Convert to datetime
    
    # Rename "host_since" to "days_as_host"
    df.rename(columns={date_col: 'days_as_host'}, inplace = True)
    
    # Calculate the number of days from reference date
    df['days_as_host'] = (reference_date - df['days_as_host']).dt.days

    return df
# Reference day: 05/09/2024 (since this is when it was last scraped)
# Relevant column: "host_since"

In [5]:
# Function to convert the attributes with "t" and "f" to "True" and "False"

def convert_to_boolean(df: pd.DataFrame, boolean_cols: list):
    for col in boolean_cols:
        df[col] = df[col].replace({'f':False, 't': True})        
    # Removing any listing with NaN values 
    df = df.dropna(subset=boolean_cols)
    return df

In [6]:
# Function to convert the location of the host, to "True/False" if they are in or outside of New York

def is_host_in_NY(df: pd.DataFrame, host_location: str):
    
    # Convert to true or false
    df.loc[:, host_location] = df[host_location].apply(lambda x: True if 'New York' in str(x) or 'NY' in str(x) else False)
    
    # Rename column
    df.rename(columns={host_location: 'host_in_NY'}, inplace=True)
    return df

In [7]:
# Function to convert lists to number of amenities of listing and verifications from host
def num_elements_in_list(df: pd.DataFrame, list_cols: list):
    for col in list_cols:
        df.loc[:, col] = df[col].apply(lambda x: len(eval(x)) if isinstance(x, str) else 0)
    return df

In [8]:
#Function to identify which type of bathroom the listing has and how many
def number_and_type_bathroom(df: pd.DataFrame, bathroom_text: str):
    # Drop NaN values
    df = df.dropna(subset=[bathroom_text])
    
    # Initialize columns for private and shared bathrooms
    df['private_bathroom'] = 0
    df['shared_bathroom'] = 0
    
    # Function to extract number of bathrooms and type
    def extract_bathroom_info(text):
        text = text.lower()
        if 'shared half-bath' in text:
            return (0, 0.5)
        elif 'private half-bath' in text or 'half-bath' in text:
            return (0.5, 0)
        elif 'shared' in text:
            num_baths = float(text.split()[0])
            return (0, num_baths)
        elif 'private' in text:
            num_baths = float(text.split()[0])
            return (num_baths, 0)
        else:
            num_baths = float(text.split()[0])
            return (num_baths, 0)
    
    # Apply the function to each row
    df[['private_bathroom', 'shared_bathroom']] = df[bathroom_text].apply(lambda x: pd.Series(extract_bathroom_info(x)))
    
    return df

In [11]:
#The function to obtain integer value of the acceptance and response rates of the customer
def get_rates_categories(col):
    int_representation = int(col[:-1])
    return int_representation

In [9]:
# Using the functions to clean the data
text_cols = ["name", "description", "neighborhood_overview", "host_about"]
boolean_cols = ["host_is_superhost", "host_has_profile_pic", "host_identity_verified", "instant_bookable"]
date_col = "host_since"
reference_date = "2024-09-05"
host_loc = "host_location"
list_cols = ["host_verifications", "amenities"]

df = df[df['source'] != 'previous scrape'] # !! Removing unavailable listings !!
df = text_to_len(df, text_cols)
df = days_as_host(df, date_col, reference_date)
df = convert_to_boolean(df, boolean_cols)
df = is_host_in_NY(df, host_loc)
df = num_elements_in_list(df, list_cols)
df = df.dropna(subset=['bedrooms']) # Drop rows where 'bedrooms' is NaN as we have no other information about number of beds
df = df.dropna(subset=['beds']) # Drop rows where 'bedrooms' is NaN as we have no other information about number of beds
df = df.dropna(subset=['price']) # Drop rows where price is NaN
df['price'] = df['price'].str.replace('$', '').str.replace(',', '').astype(float) # Convert str dollar price to numerical price


/var/folders/4m/s8hcw8_d02zgx8npr9k9fh4m0000gn/T/ipykernel_40499/2492182528.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col] = df[col].replace({'f':False, 't': True})


In [10]:
# LABEL or ONE-HOT ENOCDING
df = number_and_type_bathroom(df, 'bathrooms_text') # Now we have "private_bathroom" and "shared_bathroom"
df = pd.get_dummies(df, columns=['room_type'], prefix='room_type') # Now we have 'room_type_Entire home/apt', 'room_type_Hotel room',
                                                                    # 'room_type_Private room', 'room_type_Shared room' 

/var/folders/4m/s8hcw8_d02zgx8npr9k9fh4m0000gn/T/ipykernel_40499/1964521205.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['private_bathroom'] = 0
/var/folders/4m/s8hcw8_d02zgx8npr9k9fh4m0000gn/T/ipykernel_40499/1964521205.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['shared_bathroom'] = 0
/var/folders/4m/s8hcw8_d02zgx8npr9k9fh4m0000gn/T/ipykernel_40499/1964521205.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,c

In [12]:
df = df.dropna()
df["host_response_rate_int"] = df["host_response_rate"].apply(lambda x: get_rates_categories(x))
df["host_acceptance_rate_int"] = df["host_acceptance_rate"].apply(lambda x: get_rates_categories(x))

In [13]:
df.to_csv("data_cleansed.csv", index=False)

In [14]:
df

,id,source,name,description,neighborhood_overview,host_id,host_name,days_as_host,host_in_NY,host_about,...,instant_bookable,calculated_host_listings_count,private_bathroom,shared_bathroom,room_type_Entire home/apt,room_type_Hotel room,room_type_Private room,room_type_Shared room,host_response_rate_int,host_acceptance_rate_int
1,572612125615500056,city scrape,10,76,0,358089614,Joshua,1505,True,0,...,True,2,0.0,1.0,False,False,True,False,100,94
2,45267941,city scrape,5,0,0,39162543,Jeff,3334,True,0,...,False,3,0.0,1.0,False,False,True,False,100,77
3,838141198693830649,city scrape,4,0,0,148571080,David,2562,False,63,...,False,5,2.0,0.0,True,False,False,False,0,0
4,1082660771919357919,city scrape,4,88,0,394869975,Betty,1255,True,0,...,False,3,2.5,0.0,True,False,False,False,78,33
6,1020282701018874374,city scrape,3,52,0,483056418,Kristina,696,True,58,...,False,24,1.0,0.0,False,False,True,False,100,78
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37530,633168575271694958,city scrape,7,100,15,6324250,Linh,4136,False,20,...,False,1,1.0,0.0,True,False,False,False,100,96
37532,1124574711796175408,city scrape,3,87,0,42325550,Sherveal,3301,True,0,...,False,1,1.0,0.0,True,False,False,False,100,100
37535,704255897463350082,city scrape,6,62,33,476666525,Anadis,742,False,110,...,False,1,1.0,0.0,True,False,False,False,100,100
37537,783387491435863626,city scrape,12,66,0,491695460,Prince,629,True,0,...,False,1,2.0,0.0,True,False,False,False,100,96
